# Loading Data

In [137]:
# Importing packages
import numpy as np
import pandas as pd

import re 
import warnings
from unidecode import unidecode

## Regions

In [138]:
# Read regions table
regions = pd.read_excel("data/cz_regions.xlsx", header=None)
# Rename column
regions.rename(columns={regions.columns[0] : "region"}, inplace=True)
# Remove special characters
regions["regions_base"] = regions["region"].apply(lambda x: unidecode(str(x))) 
# Get only names of each region
regions["name_base"] = regions["regions_base"].apply(lambda x: " ".join([word for word in x.split() if word.upper() != "KRAJ"]))
# Manually set Ceska republika have name_base="CZ" and Prague to have name_base = "Praha"
regions.loc[0, "name_base"] = "CZ"
regions.loc[1, "name_base"] = "Praha"

In [139]:
regions

,region,regions_base,name_base
0,Česká republika,Ceska republika,CZ
1,Hlavní město Praha,Hlavni mesto Praha,Praha
2,Středočeský kraj,Stredocesky kraj,Stredocesky
3,Jihočeský kraj,Jihocesky kraj,Jihocesky
4,Plzeňský kraj,Plzensky kraj,Plzensky
5,Karlovarský kraj,Karlovarsky kraj,Karlovarsky
6,Ústecký kraj,Ustecky kraj,Ustecky
7,Liberecký kraj,Liberecky kraj,Liberecky
8,Královéhradecký kraj,Kralovehradecky kraj,Kralovehradecky
9,Pardubický kraj,Pardubicky kraj,Pardubicky


## Housing

In [140]:
housing_dfs= []
with warnings.catch_warnings():
    # Catch openpyxl warnings
    warnings.filterwarnings("ignore", category=UserWarning, module=re.escape('openpyxl.styles.stylesheet'))
    
    # open years from 2006 until 2019
    for i in range(6, 20):
        suf = str(i) if i >= 10 else '0'+str(i)
        df = pd.read_excel(f"data/housing_indices/CEN{suf}.xlsx", skiprows=8, skipfooter=6, usecols=lambda x: 'Unnamed: 0' not in x, sheet_name="DATA")
        df.rename(columns={df.columns[0] : "region", 
                        df.columns[1] : "total", 
                        df.columns[2] : "family_houses", 
                        df.columns[3] : "apartments",
                        df.columns[4] : "apartment_buildings",
                        df.columns[5] : "plots"}, inplace=True)
        df.insert(0, "year", "20" + suf)        
        housing_dfs.append(df)
    
    # open years from 2020 until 2024 - different due to change of table structure
    for i in range(21, 25):
        suf = str(i) if i >= 10 else '0'+str(i)
        df = pd.read_excel(f"data/housing_indices/CEN{suf}.xlsx", skiprows=8, skipfooter=5, usecols=lambda x: 'Unnamed: 0' not in x, sheet_name="DATA")
        df.insert(0, "year", "20" + suf)
        df.rename(columns={df.columns[1] : "region", 
                        df.columns[6] : "family_houses", 
                        df.columns[11] : "apartments"}, inplace=True)
        df = df.loc[:,~df.columns.str.startswith('Q')]
        housing_dfs.append(df)
        
# Concat the years
housing = pd.concat(housing_dfs, ignore_index=True)

### Fix Indexing

Due to different base years throughout the data series, the indexes have to be adjusted such that all years reflect the year 2006 has index 100.

In [141]:
# Import edge years where indexing changes
# 2005-2010: 2005=100
# 2011-2024: 2010=100

with warnings.catch_warnings():
    # Catch openpyxl warnings
    warnings.filterwarnings("ignore", category=UserWarning, module=re.escape('openpyxl.styles.stylesheet'))

    # 2011 (_o suffix = data compared to the same period in the previous year)
    housing11 = pd.read_excel("data/housing_indices/CEN11_o.xlsx", sheet_name="DATA", skiprows=8, skipfooter=6, usecols=lambda x: 'Unnamed: 0' not in x)
    housing11.rename(columns={housing11.columns[0] : "region", 
                    housing11.columns[1] : "total", 
                    housing11.columns[2] : "family_houses", 
                    housing11.columns[3] : "apartments",
                    housing11.columns[4] : "apartment_buildings",
                    housing11.columns[5] : "plots"}, inplace=True)
    housing11.insert(0, "year", "2011")   
    

In [142]:
# Make sure year is int
housing["year"] = housing["year"].astype(int)
housing11["year"] = housing11["year"].astype(int)

# Rebase 2006 to 100
for region in housing["region"].unique():
    base_2006 = housing.loc[(housing["year"] == 2006) & (housing["region"] == region), ["family_houses", "apartments"]]

    if not base_2006.empty:
        factor = 100 / base_2006.values  # scaling so that 2006 == 100
        mask = (housing["region"] == region) & (housing["year"].between(2006, 2010))
        housing.loc[mask, ["family_houses", "apartments"]] = (
            housing.loc[mask, ["family_houses", "apartments"]].values * factor
        )

# Rebase 2011 using housing11 (growth vs 2010)
for region in housing["region"].unique():
    val_2010 = housing.loc[(housing["year"] == 2010) & (housing["region"] == region), ["family_houses", "apartments"]]
    factor_2011 = housing11.loc[(housing11["year"] == 2011) & (housing11["region"] == region), ["family_houses", "apartments"]]

    if not val_2010.empty and not factor_2011.empty:
        # factor_2011 is e.g. 102.1 → means +2.1% → scale = 1.021
        scale = factor_2011.values / 100
        new_2011 = val_2010.values * scale
        housing.loc[(housing["year"] == 2011) & (housing["region"] == region), ["family_houses", "apartments"]] = new_2011

# Chain forward 2012–2024
# Each year is expressed as index relative to 2010=100, so now we anchor them to the rebased 2010
for region in housing["region"].unique():
    base_2010 = housing.loc[(housing["year"] == 2010) & (housing["region"] == region), ["family_houses", "apartments"]]
    if not base_2010.empty:
        factor_2010 = housing.loc[(housing["year"] == 2010) & (housing["region"] == region), ["family_houses", "apartments"]].values / 100
        mask = (housing["region"] == region) & (housing["year"] >= 2011)
        housing.loc[mask, ["family_houses", "apartments"]] = (
            housing.loc[mask, ["family_houses", "apartments"]].values * factor_2010
        )


In [143]:
# Left join region name_base
housing = pd.merge(housing, regions, how="left", on="region")

In [144]:
# show sample of data
housing.sample(10)

,year,region,total,family_houses,apartments,apartment_buildings,plots,regions_base,name_base
259,2024,ČR bez Prahy,NaN,242.300000,266.800000,NaN,NaN,NaN,NaN
201,2019,Ústecký kraj,131.3,165.857656,162.835798,145.8,137.5,Ustecky kraj,Ustecky
224,2021,Zlínský kraj,NaN,239.356196,291.400000,NaN,NaN,Zlinsky kraj,Zlinsky
200,2019,Karlovarský kraj,149.5,168.491007,163.755169,174.9,134.0,Karlovarsky kraj,Karlovarsky
94,2012,Plzeňský kraj,100.8,119.781183,121.286239,95.9,102.9,Plzensky kraj,Plzensky
156,2016,Ústecký kraj,102.6,141.667297,114.331518,108.7,107.1,Ustecky kraj,Ustecky
22,2007,Liberecký kraj,134.0,114.380952,141.262580,129.8,136.9,Liberecky kraj,Liberecky
174,2017,Pardubický kraj,111.7,127.400000,153.378720,112.9,121.9,Pardubicky kraj,Pardubicky
123,2014,Jihočeský kraj,104.8,119.778771,125.938272,111.9,107.4,Jihocesky kraj,Jihocesky
256,2023,Zlínský kraj,NaN,273.567147,330.322529,NaN,NaN,Zlinsky kraj,Zlinsky


## Wages

In [155]:
# Read wages data for CZ region
cz_wages = pd.read_excel("data/wages/MZDCZ00_24.xlsx", sheet_name="DATA", skiprows=6, skipfooter=7, usecols=lambda x: "Unnamed: 0" not in x)
# Rename cols
cz_wages = cz_wages.rename(columns={cz_wages.columns[0] : "year",
                        cz_wages.columns[1] : "avg_no_empl_adj", # adjusted to full time equivalent
                        cz_wages.columns[2] : "avg_month_wage_adj", # adjusted to full time equivalent
                        cz_wages.columns[3] : "avg_no_empl_hc", # headcount
                        cz_wages.columns[4] : "avg_month_wage_hc" # headcount
                        })

# Make sure year is int
cz_wages["year"] = cz_wages["year"].astype(int)
cz_wages.insert(0, "region", "CZ")
# Order by year in ascending
cz_wages = cz_wages.sort_values(["year"]).reset_index().drop(["index"],axis=1)


In [156]:
# Import regional wages
wages_regions = []
with warnings.catch_warnings():
    # Catch openpyxl warnings
    warnings.filterwarnings("ignore", category=UserWarning, module=re.escape('openpyxl.styles.stylesheet'))
        
    for i in regions["name_base"]:
        if i != "CZ": # --- ion like this here, make it cleaner X)))
            for j in ["00_10", "11_24"]:
                df = pd.read_excel(f"data/wages/MZD{i+j}.xlsx", sheet_name="DATA", skiprows=6, skipfooter=8 if j=="11_24" else 6, usecols = lambda x: "Unnamed: 0" not in x)
                df.rename(columns={df.columns[0] : "year",
                                        df.columns[1] : "avg_no_empl_adj", # adjusted to full time equivalent
                                        df.columns[2] : "avg_month_wage_adj", # adjusted to full time equivalent
                                        df.columns[3] : "avg_no_empl_hc", # headcount
                                        df.columns[4] : "avg_month_wage_hc" # headcount
                                        }, inplace=True)
                df.insert(0, "region", i)  
                # get rid of the notes in years
                df["year"] = df['year'].astype(str).str.slice(0, 4).astype(int)
                wages_regions.append(df)

# Concat regions
wages_regions = pd.concat(wages_regions, ignore_index=True)

In [157]:
# merge wages for regions and aggregate
wages = pd.concat([cz_wages, wages_regions], ignore_index=True)

# order by region and year
wages = wages.sort_values(["region", "year"]).reset_index().drop(["index"],axis=1)

# filter to years [2006,2024]
# wages = wages[wages['year'].astype(int).between(2006, 2024)]

In [159]:
wages[wages["region"] == "Liberecky"]

,region,year,avg_no_empl_adj,avg_month_wage_adj,avg_no_empl_hc,avg_month_wage_hc
125,Liberecky,2000,157.600000,12185.000000,162.100000,11850.000000
126,Liberecky,2001,160.300000,13222.000000,165.200000,12831.000000
127,Liberecky,2002,153.700000,14113.000000,158.600000,13677.000000
128,Liberecky,2003,151.600000,14962.000000,156.300000,14512.000000
129,Liberecky,2004,150.200000,15824.000000,154.700000,15360.000000
130,Liberecky,2005,154.300000,16691.000000,158.900000,16205.000000
131,Liberecky,2006,153.900000,17747.000000,158.600000,17223.000000
132,Liberecky,2007,153.700000,19025.000000,158.700000,18432.000000
133,Liberecky,2008,153.000000,20373.000000,157.700000,19762.000000
134,Liberecky,2009,139.500000,20953.000000,143.900000,20310.000000


### Fix Indexing

Due to nominal values in the data series, the values are reindexed such that year 2006 has value 100 for every region and column.

In [ ]:
# --- CONFIG
base_year = 2006
cols_to_index = ['avg_no_empl_adj', 'avg_month_wage_adj', 'avg_no_empl_hc', 'avg_month_wage_hc']

# --- 0) Basic type safety
# convert year to int (nullable Int64) and ensure region is string
wages['year'] = pd.to_numeric(wages['year'], errors='coerce').astype('Int64')
wages['region'] = wages['region'].astype(str)

# --- 1) Filter years 2006-2024 and copy
w = wages[wages['year'].between(2006, 2024)].copy()

# --- 2) Remove exact duplicate rows, then ensure single row per (region, year)
w = w.drop_duplicates()
w = w.sort_values(['region', 'year']).drop_duplicates(['region', 'year'], keep='first').reset_index(drop=True)

# --- 3) Ensure numeric columns
for c in cols_to_index:
    w[c] = pd.to_numeric(w[c], errors='coerce')

# --- 4) Build per-region 2006 base (use mean in case of multiple 2006 rows)
base_df = (
    w[w['year'] == base_year]
    .groupby('region')[cols_to_index]
    .mean()
    .reset_index()
    .rename(columns={c: f'{c}_base' for c in cols_to_index})
)

# --- 5) Merge base back into main table
w = w.merge(base_df, on='region', how='left')

# --- 6) Compute indexed values (2006 = 100) and overwrite original columns
for c in cols_to_index:
    base_col = f'{c}_base'
    # avoid division by zero / NaN
    w[c] = np.where(w[base_col].notna() & (w[base_col] != 0),
                    (w[c] / w[base_col]) * 100,
                    np.nan)

# --- 7) Drop helper base columns
w = w.drop(columns=[f'{c}_base' for c in cols_to_index])

# --- 8) Sanity checks & warnings
# Regions that have NaN in the first base column are missing 2006 base
missing_regions = sorted(w[w[cols_to_index[0]].isna()]['region'].unique().tolist())
if missing_regions:
    print("Warning: the following regions are missing 2006 base values (check spelling/duplicates or data completeness):")
    print(missing_regions)

# Replace wages DataFrame (optional) - if you want to keep the old one, skip this
wages_rebased = w.copy()


In [149]:
# Left join region name_base
# wages_rebased = 
wages_rebased.rename(columns={"region": "name_base"}, inplace=True)
wages_rebased = pd.merge(wages_rebased, regions, how="left", on="name_base")

In [150]:
# show sample
wages_rebased.sample(10)

,name_base,year,avg_no_empl_adj,avg_month_wage_adj,avg_no_empl_hc,avg_month_wage_hc,region,regions_base
111,Liberecky,2022,93.472099,203.159463,94.896987,200.086696,Liberecký kraj,Liberecky kraj
113,Liberecky,2024,94.425898,234.121074,96.255189,229.646094,Liberecký kraj,Liberecky kraj
253,Vysocina,2012,95.041783,126.450366,94.637053,126.977913,Kraj Vysočina,Kraj Vysocina
4,CZ,2010,96.218455,122.091477,96.149496,122.171108,Česká republika,Ceska republika
23,Jihocesky,2010,94.181502,120.479291,94.070648,120.579374,Jihočeský kraj,Jihocesky kraj
7,CZ,2013,94.980813,128.082472,94.984018,128.072124,Česká republika,Ceska republika
171,Plzensky,2006,100.000000,100.000000,100.000000,100.000000,Plzeňský kraj,Plzensky kraj
124,Moravskoslezsky,2016,99.625906,136.514971,99.863605,136.205459,Moravskoslezský kraj,Moravskoslezsky kraj
132,Moravskoslezsky,2024,97.510181,223.303825,99.451055,218.944314,Moravskoslezský kraj,Moravskoslezsky kraj
126,Moravskoslezsky,2018,102.863033,156.778721,102.844750,156.805488,Moravskoslezský kraj,Moravskoslezsky kraj
